## Planificación de la producción

#### Problema CQM c/ reformulación Qubo


Una empresa de recambios industriales produce dos tipos de baterías, A y B. 

- Su producción semanal debe ser de al menos 30 baterías en total y el número de baterías de tipo B no puede
superar en más de 10 unidades a las fabricadas del tipo A. 

- Cada batería de tipo A tiene unos gastos de producción de 150 euros y cada batería de tipo B de 100 euros, disponiendo de un máximo de 6000euros a la semana para costes de producción.

- Si la empresa vende todo lo que produce y cada batería de tipo A se vende a  130euros y la de tipo B a 140 euros, 

1. ¿cuántas baterías de cada tipo tendrán que producir a la semana para que el beneficio total sea máximo?. 

2. ¿Cuál es el beneficio semanal?


<b>Planteamiento algebraico:</b>

    - x1, x2 ; variables bat A y bat B

    - Función objetivo: minimizar{f(x1,x2)} = -130*x1 - 140*x2

- Restricciones:

    - x1 + x2 >=30
    - x2 - x1 <= 10
    - 15*x1 + 10*x2 <= 600



In [151]:
# Recursos
import dimod
from dimod import ConstrainedQuadraticModel, ExactCQMSolver, Integer
from dwave.system import DWaveSampler, EmbeddingComposite

# Funciones auxiliares

def sol_factibles(sampleset,n=None):
    # Resultados
    
    subset=sampleset.filter(lambda s: s.is_feasible).aggregate()
    
    if not n:
        print(f"\nLas {len(subset)} soluciones factibles al problema son:\n")
    else:
        print(f"\nImprimiendo las {n}/{len(subset)} primeras soluciones factibles:\n")
    
    print(subset.slice(n))
    
    
def sol_problema(result):
    # Agregación de los n reads
    samples = []
    ocurrencias = []

    for s in result.data():
        samples.append(invert(s.sample))
        ocurrencias.append(s.num_occurrences)

    sampleset = dimod.SampleSet.from_samples_cqm(samples,cqm,num_occurrences=ocurrencias)
    return(sampleset)

In [153]:
# Definición del problema CQM

x1 = Integer('Xa',upper_bound=40)
x2 = Integer('Xb',upper_bound=40)

cqm = ConstrainedQuadraticModel()


# Función objetivo: minimizar beneficio

cqm.set_objective(-13*x1 - 14*x2)

# Restricciones

cqm.add_constraint(x1 + x2 >= 30,"Producción 1: Xa + Xb >=30")
cqm.add_constraint(x2 - x1 <= 10,"Producción 2: Xb - Xa <= 10")
cqm.add_constraint(15*x1 + 10*x2 <= 600,"Costes:150Xa + 100Xb <= 6000")
print("Planteamiento problema CQM")
print("==========================")
print(cqm)

Planteamiento problema CQM
Constrained quadratic model: 2 variables, 3 constraints, 8 biases

Objective
  -13*Integer('Xa') - 14*Integer('Xb')

Constraints
  Producción 1: Xa + Xb >=30: Integer('Xa') + Integer('Xb') >= 30.0
  Producción 2: Xb - Xa <= 10: Integer('Xb') - Integer('Xa') <= 10.0
  Costes:150Xa + 100Xb <= 6000: 15*Integer('Xa') + 10*Integer('Xb') <= 600.0

Bounds
  0.0 <= Integer('Xa') <= 40.0
  0.0 <= Integer('Xb') <= 40.0



### Solución por fuerza bruta

In [154]:
result = ExactCQMSolver().sample_cqm(cqm)

In [155]:
solCPU=sol_factibles(result,10)


Imprimiendo las 10/376 primeras soluciones factibles:

  Xa Xb energy num_oc. is_sat. is_fea.
0 20 30 -680.0       1 arra...    True
1 20 29 -666.0       1 arra...    True
2 21 28 -665.0       1 arra...    True
3 22 27 -664.0       1 arra...    True
4 19 29 -653.0       1 arra...    True
5 20 28 -652.0       1 arra...    True
6 21 27 -651.0       1 arra...    True
7 22 26 -650.0       1 arra...    True
8 23 25 -649.0       1 arra...    True
9 24 24 -648.0       1 arra...    True
['INTEGER', 10 rows, 10 samples, 2 variables]



<b>Resultados:</b>

- El importe semanal máximo por ventas: 6800€

- Coste producción semanal : 6000€

- Beneficio máximo: 600€

### Solución con QPU

In [156]:
#reformulación qubo y sampling

fl=20 # factor penalizador de lagrange

qubo, invert = dimod.cqm_to_bqm(cqm, lagrange_multiplier = fl)

sampler = EmbeddingComposite(DWaveSampler())
result = sampler.sample(qubo, num_reads=4000)

In [148]:
sampleset=sol_problema(result)
solQPU=sol_factibles(sampleset,5)


Imprimiendo las 5/358 primeras soluciones factibles:

  Xa Xb energy num_oc. is_sat. is_fea.
0 20 30 -680.0       3 arra...    True
1 20 29 -666.0       4 arra...    True
2 21 28 -665.0       3 arra...    True
3 22 27 -664.0       5 arra...    True
4 19 29 -653.0       3 arra...    True
['INTEGER', 5 rows, 18 samples, 2 variables]




<b>Analizando los resultados del sampler QPU</b>

- Se ha formulado un problema Qubo desde un problema CQM

- El espacio de soluciones factibles es similar al de fuerza bruta (sobre 350)

- Se han replicado los valores óptimos obtenidos por fuerza bruta

- Se necesitaron entre 3000-4000 samples para objetivo mínimo con moda sup > 3 (un umbral sensato)
